In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import re

In [2]:
warnings.filterwarnings('ignore')

advocacy = pd.read_csv('~/Documents/StatisticsBYU/426/426-Final-Project/AdvocacyRecordsDatab_DATA_LABELS_2020-11-13_0915.csv')

In [3]:
len(advocacy['Case Number'].unique()) #how many unique patients

343

### Data Cleaning

In [4]:
# Dropping unneeded columns
advocacy_dirty = advocacy.iloc[:, :-50] #drop last 50 columns

pat_info = advocacy_dirty.drop(columns=['Best way to contact (choice=Phone)', 
                         'Best way to contact (choice=Email)', 
                         'Best way to contact (choice=Text Message)', 
                         'Best way to contact (choice=Other)', 
                         'Describe other',
                         'New client?', 
                         'Intake', 
                         'Would the client like to be added to the CSW email list?',
                         'How did you hear about us? (choice=Self)',
                         'How did you hear about us? (choice=OEO)',
                         'How did you hear about us? (choice=ODOS)',
                         'How did you hear about us? (choice=Advocate Website (intake form))',
                         'How did you hear about us? (choice=UCC)',
                         'How did you hear about us? (choice=WRC)',
                         'How did you hear about us? (choice=Friend)',
                         'How did you hear about us? (choice=HR)',
                         'How did you hear about us? (choice=Unknown)',
                         'How did you hear about us? (choice=Other)', 
                         'Please specify other.1',
                         'Other', # gender
                         'Other.1', # sexuality
                         'Other.2', # ethnicity
                         'Other.3', # other vet status
                         'Other.4', # immigration status other
                         'Specify greek affiliation',
                         'Country of origin (if known).1',
                         'Complete?' ,
                         'Specify first language',
                         'Specify athletics affiliation',
                         'Please specify other', 
                         'Has TSA Informed Consent Form been signed and received from the client?'])

pat_info.shape

(2231, 208)

In [5]:
# case type
cases = advocacy_dirty.iloc[:, [0, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]]
cases.head()

case_type = pd.melt(cases, id_vars=['Case Number'], 
        value_vars=['Case type (choice=Domestic Violence)', 
                    'Case type (choice=Dating Violence)', 
                    'Case type (choice=Sexual Assault)', 
                    'Case type (choice=Rape)', 
                    'Case type (choice=Stalking)', 
                    'Case type (choice=Sexual Harassment)', 
                    'Case type (choice=Trafficking)', 
                    'Case type (choice=Family Violence)', 
                    'Case type (choice=Unknown)', 
                    'Case type (choice=Other)'], 
        var_name = 'Case Type',
        value_name='Value')

case_check = case_type[case_type['Value'] == 'Checked'] # only keep when it is 'checked'
case_check['True Type'] = case_check['Case Type'].apply(lambda x: re.split('=',x)[1][0:-1]) # split and only keep affiliation, remove last parentheses
ct_df = case_check.drop(columns=['Case Type', 'Value']) #drop unneeded columns
ct_clean = ct_df.drop_duplicates('Case Number')
ct_clean.head()

,Case Number,True Type
1,1,Domestic Violence
52,5,Domestic Violence
69,8,Domestic Violence
78,11,Domestic Violence
117,17,Domestic Violence


In [6]:
# combine some case types
def case_combine(case):
    if (str(case) == 'Domestic Violence') or (str(case) == 'Dating Violence') or (str(case) == 'Family Violence'):
        return('Violence')
    elif str(case) == 'Sexual Assault' or str(case) == 'Rape':
        return('Sexual Assault')
    elif str(case) == 'Stalking' or str(case) == 'Sexual Harassment':
        return('Sexual Harassment')
    else:
        return('Other')

In [7]:
ct_clean['Case Type Modified'] = ct_clean['True Type'].apply(lambda x: case_combine(x))

In [8]:
# take the first rows of each case number with patient info
first_rows = advocacy_dirty[pd.isna(advocacy_dirty['Repeat Instance'])]
len(advocacy_dirty['Case Number'].unique()) #number of unique patients

343

In [9]:
# count the number of times a patient came to the office
case_count = advocacy_dirty.groupby('Case Number')['Case Number'].count()
case_df = pd.DataFrame(case_count)
case_df.columns = ['Case Count']
case_df.reset_index()

,Case Number,Case Count
0,1,30
1,2,7
2,3,4
3,4,10
4,5,5
...,...,...
338,340,3
339,341,2
340,342,3
341,343,2


In [10]:
# drop any columns that were completely NaNs
patient_info = first_rows.loc[:,first_rows.count()>0]

cols = first_rows.count()>0

In [11]:
pat_info = first_rows.loc[:,first_rows.count()>0]
pat_info.shape

(343, 63)

In [12]:
pat_info.iloc[:, 8:15]

,University Affiliation (choice=Student),University Affiliation (choice=Employee),University Affiliation (choice=Faculty),University Affiliation (choice=Secondary Survivor),University Affiliation (choice=Secondary Survivor (non-campus member)),University Affiliation (choice=Former Student),University Affiliation (choice=Other)
0,Checked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Unchecked
30,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
37,Checked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
41,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
51,Checked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
...,...,...,...,...,...,...,...
2219,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
2222,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
2224,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked
2227,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked


In [13]:
# look at what is notnull
#pat_info[pat_info['Limited English proficiency (from advocate\'s observation/discretion) '].notnull()]['Limited English proficiency (from advocate\'s observation/discretion) ']

In [14]:
import re
# clean up the different pronoun levels
# pat_info.Pronouns.unique() # she, he, they, not dicslosed

gender = str()
gender_list = []

def gender_ident(x):
    if re.search('she', str(x)) or re.search('her', str(x)):
        gender = 'She/Her'
    elif re.search('he', str(x)) or re.search('him', str(x)):
        gender = 'He/Him'
    elif re.search('they', str(x)) or re.search('them', str(x)):
        gender = 'They/Them'
    else:
        gender = 'NaN'
    return(gender) #do return not print so you can save the new gender

#add cleaned up pronouns as assumed gender
pat_info['gender'] = pat_info['Pronouns'].apply(lambda x : gender_ident(x))



In [15]:
# fix university affiliation

# create dataframe of univ affiliation column - may need to change iloc 
univ_aff = pd.melt(pat_info.iloc[:,[0,8,9,10,11,12,13,14,15]],id_vars='Case Number', value_vars=[
                                          'University Affiliation (choice=Student)',
                                          'University Affiliation (choice=Employee)', 
                                          'University Affiliation (choice=Faculty)', 
                                          'University Affiliation (choice=Secondary Survivor)', 
                                          'University Affiliation (choice=Secondary Survivor (non-campus member))', 
                                          'University Affiliation (choice=Former Student)'], 
                   var_name='Univ Affiliation', value_name = 'Association')
uni_check = univ_aff[univ_aff['Association'] == 'Checked'] # only keep when it is 'checked'
uni_check['Relation'] = uni_check['Univ Affiliation'].apply(lambda x: re.split('=',x)[1][0:-1]) # split and only keep affiliation, remove last parentheses
uni_relation = uni_check.drop(columns=['Univ Affiliation', 'Association']) #drop unneeded columns

uni_relation.head()

,Case Number,Relation
0,1,Student
2,3,Student
3,4,Student
4,5,Student
5,6,Student


In [16]:
df = pat_info.drop(columns=['University Affiliation (choice=Employee)', 
                       'University Affiliation (choice=Faculty)', 
                       'University Affiliation (choice=Secondary Survivor)', 
                       'University Affiliation (choice=Secondary Survivor (non-campus member))', 
                       'University Affiliation (choice=Former Student)', 
                       'University Affiliation (choice=Other)',
                       'University Affiliation (choice=Student)', 
                       'Pronouns'])
df.shape
pat_almost = df.merge(uni_relation, on='Case Number')  # merge with original dataframe
pat_almost.shape # why are there more than 343? need to drop duplicates? how to combine if there are duplicates?

(405, 57)

In [17]:
# fix physical abuse experience
phys_exp = pd.melt(pat_almost.iloc[:,[0,25,26,27,28,29]],id_vars='Case Number', value_vars=[
                                          'Have you experienced physical abuse and/or assault? (choice=Unsure)',
                                          'Have you experienced physical abuse and/or assault? (choice=Yes)'], 
                   var_name='Question', value_name = 'Physical Abuse Experience')


phys_check = phys_exp[phys_exp['Physical Abuse Experience'] == 'Checked'] # only keep when it is checked
phys_check['Physical Abuse Exp'] = phys_check['Question'].apply(lambda x: re.split('=',x)[1][0:-1]) # split and only keep affiliation, remove last parentheses
phys_relation = phys_check.drop(columns=['Question', 'Physical Abuse Experience']) #drop unneeded columns


In [18]:
pat_almost.merge(phys_relation, on='Case Number')

pat_complete = pat_almost.drop(columns=['Have you experienced physical abuse and/or assault? (choice=Unsure)',
                         'Have you experienced physical abuse and/or assault? (choice=Yes)',
                         'Have you experienced physical abuse and/or assault? (choice=No)', 
                         'Have you experienced physical abuse and/or assault? (choice=Prefer not to disclose)'])

In [19]:
def advocate_change(name):
    if name == 'Ellie Goldberg':
        return('A')
    elif name == 'Darrah Jones':
        return('B')
    elif name == 'Benta Opiyo':
        return('C')
    elif name == 'Mahalia Lotz':
        return('D')
    elif name == 'Maria Skedros':
        return('E')
    elif name == 'Cody Lockyer':
        return('F')
    elif name == 'Char Leary':
        return('G')
    else:
        return()

In [20]:
pat_complete['Advocate'] = pat_complete['Advocate'].apply(lambda x: advocate_change(x)) # save over who the advocate is

In [21]:
# tack on case count
pat_end = pat_complete.merge(case_df, on='Case Number')

In [22]:
pat_final = pat_end.drop(columns=['Best way to contact (choice=Phone)', 
                         'Best way to contact (choice=Email)', 
                         'Best way to contact (choice=Text Message)', 
                         'Best way to contact (choice=Other)', 
                         'Describe other',
                         'New client?', 
                         'Intake', 
                         'Would the client like to be added to the CSW email list?',
                         'How did you hear about us? (choice=Self)',
                         'How did you hear about us? (choice=OEO)',
                         'How did you hear about us? (choice=ODOS)',
                         'How did you hear about us? (choice=Advocate Website (intake form))',
                         'How did you hear about us? (choice=UCC)',
                         'How did you hear about us? (choice=WRC)',
                         'How did you hear about us? (choice=Friend)',
                         'How did you hear about us? (choice=HR)',
                         'How did you hear about us? (choice=Unknown)',
                         'How did you hear about us? (choice=Other)', 
                         'Please specify other.1',
                         'Other', # gender
                         'Other.1', # sexuality
                         'Other.2', # ethnicity
                         'Other.3', # other vet status
                         'Other.4', # immigration status other
                         'Specify greek affiliation',
                         'Country of origin (if known).1',
                         'Complete?' ,
                         'Specify first language',
                         'Specify athletics affiliation',
                         'Please specify other', 
                         'Has TSA Informed Consent Form been signed and received from the client?'])

pat_final.shape

(405, 23)

In [23]:
pat_last = pat_final.merge(ct_clean, on='Case Number')

In [24]:
pat_last.shape

(361, 25)

In [25]:
pat_last.to_csv('patient_data.csv')